# Drive mount

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회

/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회


# install

In [ ]:
# !pip uninstall transformers
!pip install transformers==4.36.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 92.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
# !pip install transformers==4.42.3 accelerate #gemma
!pip install --upgrade transformers==4.44.0 #llama
# !pip install --upgrade transformers
!pip install bitsandbytes
!pip install pdfplumber
!pip install -r requirements.txt
!pip install -U langchain-community
!pip install chromadb

# https://huggingface.co/snunlp/KR-SBERT-V40K-klueNLI-augSTS
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 109.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.43.1
    Uninstalling transformers-4.43.1:
      Successfully uninstalled transformers-4.43.1


# Load CSV

In [ ]:
import pandas as pd

df = pd.read_csv("train.csv")

pdfs = df['Source_path'].unique()

Questions = []
for ind, pdf in enumerate(pdfs):
  Questions.append({
      'index' : 'pdf_'+ str(ind),
      'Source' : pdf,
      'Question' : df.loc[df['Source_path']==pdf]['Question']
      })

In [ ]:
Questions[0]['index'],Questions[0]['Source'], len(Questions[0]['Question'])

('pdf_0', './train_source/1-1 2024 주요 재정통계 1권.pdf', 50)

In [ ]:
# df.loc[df['Source']=='「FIS 이슈 & 포커스」 22-3호 《재정융자사업》'][['Question','Answer']]

# Load Data

In [ ]:
DATA_PATH = "./train_source"

In [ ]:
SOURCE_PATH = "/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회/train_source/「FIS 이슈 & 포커스」 22-3호 《재정융자사업》.pdf"

In [ ]:
import argparse
import os
import numpy as np
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
# from get_embedding_function import get_embedding_function
from langchain.vectorstores.chroma import Chroma
from langchain_community.document_loaders import PDFPlumberLoader
from tqdm import tqdm


In [ ]:
# 하위 폴더 pdf 전부 LOAD
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

In [ ]:
def load_pdf_file(SOURCE_PATH):
  file_loader = PDFPlumberLoader(SOURCE_PATH)
  return file_loader.load()

In [ ]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [ ]:
documents = load_pdf_file(SOURCE_PATH)

In [ ]:
split_docs = split_documents(documents)

In [ ]:
split_documents[0]

Document(metadata={'source': '/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회/train_source/「FIS 이슈 & 포커스」 22-3호 《재정융자사업》.pdf', 'file_path': '/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회/train_source/「FIS 이슈 & 포커스」 22-3호 《재정융자사업》.pdf', 'page': 0, 'total_pages': 9, 'CreationDate': "D:20221128103324+09'00'", 'Creator': 'Adobe InDesign CS6 (Windows)', 'ModDate': "D:20221128104540+09'00'", 'Producer': 'Adobe PDF Library 10.0.1', 'Trapped': 'False'}, page_content='22-3호\nFIS\n2022.11.\nISSUE & FOCUS\n발행인 박용주 발행처 04637 서울특별시 중구 퇴계로 10(남대문로5가 537) 메트로타워 T 02)6908-8200 F 02)6312-8959\n작성 박정수 부연구위원, 우수연 연구원 기획·조정 심혜인 결산정보분석부장\n재정융자사업\n1 들어가며\n2 재정융자사업의 개념과 의의\n3 2023년도 예산안 재정융자사업 현황\n4 재정융자사업의 주요 현안\n5 나가며')

In [ ]:
meta_dict = []
page_content = []
for doc in split_documents:
    meta_dict.append(doc.metadata)
    page_content.append(doc.page_content)

# page_content

# Embedding

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
def get_embedding_funtion(chunks):
  model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
  embeddings = []
  for chunk in tqdm(chunks):
    embeddings.append(model.encode(chunk))
  return np.array(embeddings)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings = get_embedding_funtion(page_content)

100%|██████████| 28/28 [00:01<00:00, 19.58it/s]


# Vector DB

In [ ]:
import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection("my_collection")

In [ ]:
metadatas = []
ids = []

for i, article in enumerate(tqdm(split_documents)):
  metadatas.append({
      'source':article.metadata['source'],
      'page':article.metadata['page'],
      'chunk':article.page_content
      })
  ids.append(str(i))

100%|██████████| 28/28 [00:00<00:00, 224551.65it/s]


In [ ]:
embeddings_list = embeddings.tolist()

In [ ]:
collection.add(
    embeddings=embeddings_list,
    metadatas=metadatas,
    ids=ids
)

In [ ]:
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
query = "우수연구원"
results = collection.query(
    query_embeddings = model.encode([query]),
    n_results=4
)

In [ ]:
results

{'ids': [['26', '24', '0', '22']],
 'distances': [[376.201904296875,
   424.2937927246094,
   424.6885070800781,
   426.8180236816406]],
 'metadatas': [[{'chunk': '\x07\x07\x07\x07-\x07시각화\x07정보,\x07시계열\x07분석\x07기능\x07등을\x07제공하여\x07재정정보를\x07활용한\x07다양한\x07분석\x07및\x07연구를\x07뒷받침할\x07예정\n목적으로, 우리나라를 포함해 美·日·獨·핀란드 등 총 78개 회원국과 사무국(WB+IMF)으로 구성.\n14 15',
    'page': 7,
    'source': '/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회/train_source/「FIS 이슈 & 포커스」 22-3호 《재정융자사업》.pdf'},
   {'chunk': '수행하는\x07대부분의\x07기금이\x07융자\x07지출은\x07축소하거나\x07유지하면서\x07회수\x07규모를\x07늘릴\x07전망\n\x07\x07\x07\x07-\x07기후위기에\x07대응하기\x07위한\x07기후대응기금\x07융자\x07운영\x07등,\x07융자사업으로\x07인해\x07조성된\x07기금은\x07앞으로\x07기업의\x07탄\n<표 4> 기후대응기금 개요\n소\x07감축\x07사업\x07등을\x07통해\x07지원\n국제적 중요성\n• 「기후행동\x07재무장관연합5)\x07제8차\x07장관회의」에서는\x07‘녹색\x07전환을\x07위한\x07경제정책\x07과제’를\x07주제로\x07에너지\x07\n장기적 시계열에서 재정융자의 시대적 기능과 역할을 확인할 수 있는 통계의 관리 기반 마련 등이 필요\n전환을\x07중심으로\x07재정정책\x07및\x07시장적\x07접근법,\x07규제체계\x07등\x07각국의\

In [ ]:
[result["chunk"] for result in results['metadatas'][0]]

['\x07\x07\x07\x07-\x07시각화\x07정보,\x07시계열\x07분석\x07기능\x07등을\x07제공하여\x07재정정보를\x07활용한\x07다양한\x07분석\x07및\x07연구를\x07뒷받침할\x07예정\n목적으로, 우리나라를 포함해 美·日·獨·핀란드 등 총 78개 회원국과 사무국(WB+IMF)으로 구성.\n14 15',
 '수행하는\x07대부분의\x07기금이\x07융자\x07지출은\x07축소하거나\x07유지하면서\x07회수\x07규모를\x07늘릴\x07전망\n\x07\x07\x07\x07-\x07기후위기에\x07대응하기\x07위한\x07기후대응기금\x07융자\x07운영\x07등,\x07융자사업으로\x07인해\x07조성된\x07기금은\x07앞으로\x07기업의\x07탄\n<표 4> 기후대응기금 개요\n소\x07감축\x07사업\x07등을\x07통해\x07지원\n국제적 중요성\n• 「기후행동\x07재무장관연합5)\x07제8차\x07장관회의」에서는\x07‘녹색\x07전환을\x07위한\x07경제정책\x07과제’를\x07주제로\x07에너지\x07\n장기적 시계열에서 재정융자의 시대적 기능과 역할을 확인할 수 있는 통계의 관리 기반 마련 등이 필요\n전환을\x07중심으로\x07재정정책\x07및\x07시장적\x07접근법,\x07규제체계\x07등\x07각국의\x07녹색전환\x07경험을\x07공유(2022.10.12.)\x07\n\x07재정융자는\x07대한민국\x07정부수립\x07이후부터\x07정부의\x07주요\x07기능\x07중\x07하나였으나,\x07재정융자와\x07관련된\x07통계는\x07\n• 우리나라는\x07녹색전환을\x07위한\x07공적\x07재원의\x07역할로써\x07녹색예산(green\x07budgeting)과\x07기후대응기금\x07등\x07\n시계열이\x07단절되거나\x07연장되지\x07않아,\x07시기별\x07정책\x07성과를\x07비교하고\x07향후\x07방향\x07모색에\x07활용하기\x07어려움\n재정을\x07통한\x07저탄소\x07전환\

# llama3

In [ ]:
from huggingface_hub import login
import os

login(token="token")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## llama3.1 8B

In [ ]:
# Load model directly
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)
import torch

LLAMA_3_MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)


tokenizer = AutoTokenizer.from_pretrained(LLAMA_3_MODEL_ID)
LLama3 = AutoModelForCausalLM.from_pretrained(
    LLAMA_3_MODEL_ID,
    torch_dtype="auto",
    quantization_config=quantization_config,
    device_map="auto",
)
LLama3.eval()

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

## llama3.1 70B

In [ ]:
# Load model directly
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)
import os
import torch

LLAMA_3_MODEL_ID = "meta-llama/Meta-Llama-3.1-70B-Instruct"

In [ ]:
# declare 8 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)


tokenizer = AutoTokenizer.from_pretrained(LLAMA_3_MODEL_ID)
LLama3 = AutoModelForCausalLM.from_pretrained(
    LLAMA_3_MODEL_ID,
    torch_dtype="auto",
    quantization_config=quantization_config,
    device_map="auto",
)
LLama3.eval()

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/59.6k [00:00<?, ?B/s]

...

model-00027-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00028-of-00030.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00029-of-00030.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00030-of-00030.safetensors:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

## MLP-KTLim/llama-3-Korean-Bllossom-8B

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch

LLAMA_3_MODEL_ID = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)


tokenizer = AutoTokenizer.from_pretrained(LLAMA_3_MODEL_ID)
LLama3 = AutoModelForCausalLM.from_pretrained(
    LLAMA_3_MODEL_ID,
    torch_dtype="auto",
    quantization_config=quantization_config,
    device_map="auto",
)
LLama3.eval()

# RAG

In [ ]:
def make_prompt(instruction, n_similer):

  PROMPT = f'''당신은 PDF 문서에서 정보를 검색하고 이를 바탕으로 답변하는 AI입니다.
  당신의 역할은 '정보 제공자'이고, 사용자의 역할은 '사용자'입니다.
  사용자의 질문에 대해 관련 문서에서 정보를 검색하여 명확하고 간결한 답변을 제공해야 합니다.
  답변은 무조건 한문장으로 정리하여 답변하며 질문의 대한 대답을 모두 포함해야 합니다.
  어떤 상황에서도 한문장으로 답변합니다.

  - 검색 결과의 내용을 바탕으로 사용자의 질문에 적합한 답변을 작성합니다.
  - 연도를 포함한 정보는 연도별로 요약하여 설명합니다.
  - 답변은 명확하고 간결하게 작성합니다.
  - 검색 결과의 문장을 최대한 그대로 대답합니다.

  ### 검색 결과:
  '''

  instruction = instruction

  # 백터DB 쿼리
  query = instruction

  results = collection.query(
      query_embeddings=model.encode([query]),
      n_results=n_similer
  )

  # 결과를 문자열로 변환
  results_content = "\n".join([result["chunk"] for result in results['metadatas'][0]])

  messages = [
      {"role": "system", "content": f"{PROMPT}\n\n{results_content}"},
      {"role": "user", "content": f"{instruction}"}
  ]

  return messages


In [ ]:
prompts = []
for instruction in df.loc[df['Source']=='「FIS 이슈 & 포커스」 22-3호 《재정융자사업》']['Question']:
  prompts.append(make_prompt(instruction,n_similer=5))

NameError: name 'df' is not defined

In [ ]:
answers = []
for message in tqdm(prompts):
  input_ids = tokenizer.apply_chat_template(
      message,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(LLama3.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = LLama3.generate(
      input_ids,
      max_new_tokens=512,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
      repetition_penalty=1.1,
  )

  response = outputs[0][input_ids.shape[-1]:]
  answers.append(tokenizer.decode(response, skip_special_tokens=True))

  0%|          | 0/50 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
  2%|▏         | 1/50 [00:06<05:01,  6.16s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
  2%|▏         | 1/50 [00:12<10:28, 12.83s/it]


KeyboardInterrupt: 

# F1-score

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter

In [ ]:
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):
    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

In [ ]:
def calculate_average_f1_score(true_sentences, predicted_sentences):

    total_precision = 0
    total_recall = 0
    total_f1_score = 0

    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score

    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)

    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }


In [ ]:
result = calculate_average_f1_score(df.loc[df['Source']=='「FIS 이슈 & 포커스」 22-3호 《재정융자사업》']['Answer'], answers)
print(result)

In [ ]:
df.loc[df['Source']=='「FIS 이슈 & 포커스」 22-3호 《재정융자사업》']['Answer']

,Answer
462,"재정융자사업은 정부가 자금을 민간의 사적 경제주체에 대해 대출해 주고 회수하는 활동을 말하며, 민간 금융의 역할이 미약하거나 부족한 부분에 직접 자금을 공급할 수 있는 장점이 있습니다."
463,"재정융자사업은 시장금리보다 낮은 이자율로 지원되어 자금 부족 현상을 완화해 주며, 자금 조달이 어려운 스타트업·중소기업, 담보 능력이 낮은 저소득층 등을 지원합니다."
464,"2022년 기준으로 개별 특별회계와 기금 중심의 재정융자제도를 운영하고 있으며, 2022년 현재 3개 특별회계와 29개 기금에서 융자 운영 중입니다."
465,"정부는 공공 임대주택, 중소기업 경영안정지원 등 다양한 재정융자사업을 수행하고 있다."
466,2007년부터 재정융자를 위한 특별회계 제도는 공공자금관리기금에 흡수·통합되었다.
467,"1950년대 전후 복구 및 경제 안정, 1960년대 경제개발계획의 실현, 1970년대 중화학공업화, 1980년대 경제안정화 등 각 시기별로 주요 정책 목적을 달성하기 위해 운영되었다."
468,"2023년도 예산안 기준 17개의 부처가 50개 사업을 통해 총 41.8조원 규모의 융자사업을 수행하고 있으며, 총지출 대비 6.6%를 차지합니다."
469,"2023년 예산안에서 주택도시기금의 재정융자사업이 24조 3,796억원으로 전체 융자사업의 절반 이상(58.3%)을 차지합니다."
470,"사회복지 분야로 26조 8,963억원이며, 비중은 64.3%이다."
471,"산업·중소기업및에너지 분야로 8조 4,737억원 규모이며, 전체 융자 규모 중 20.3%를 차지한다."


# Llama3.1 8B inferense

In [ ]:
from huggingface_hub import login
import os
import re
from tqdm import tqdm
import chromadb
import argparse
import pandas as pd
import numpy as np
import shutil
from collections import Counter
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
# from get_embedding_function import get_embedding_function
from langchain.vectorstores.chroma import Chroma
from langchain_community.document_loaders import PDFPlumberLoader, PyPDFium2Loader


# Load model directly
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)
import torch


In [ ]:
def extract_alphanumeric(input_string: str) -> str:
    # 정규 표현식을 사용하여 숫자와 영문자만 추출
    result = re.findall(r'[a-zA-Z0-9]+', input_string)
    # 리스트를 문자열로 결합하여 반환
    return ''.join(result)

In [ ]:
def load_pdf_file(SOURCE_PATH):
  # file_loader = PDFPlumberLoader(SOURCE_PATH)
  file_loader = PyPDFium2Loader(SOURCE_PATH)

  return file_loader.load()

In [ ]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [ ]:
from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
model = SentenceTransformer('jhgan/ko-sroberta-multitask')
def get_embedding_function(chunks):
  # model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
  model = SentenceTransformer('jhgan/ko-sroberta-multitask')
  embeddings = []
  for chunk in tqdm(chunks):
    embeddings.append(model.encode(chunk))
  return np.array(embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def make_prompt(instruction, n_similer):

  PROMPT = f'''당신은 PDF 문서에서 정보를 검색하고 이를 바탕으로 답변하는 AI입니다.
  당신의 역할은 '중앙정부 재정 정보 질의 응답 AI'이고, 사용자의 역할은 '사용자'입니다.
  다양한 재정 관련 텍스트 데이터를 활용해 주어진 질문에 대해 정확도가 높은 답변을 제공해야 합니다.
  답변은 무조건 한문장으로 정리하여 답변하며 질문의 대한 대답을 모두 포함해야 합니다.
  검색 결과만을 사용하여 정보를 검색하여 명확하고 간결한 답변을 제공해야 합니다.
  어떤 상황에서도 한문장으로 답변합니다.
  답변에 한국어만을 사용합니다.

  - 검색 결과의 내용을 바탕으로 사용자의 질문에 적합한 답변을 작성합니다.
  - 연도를 포함한 정보는 연도별로 요약하여 설명합니다.
  - 답변은 명확하고 간결하게 작성합니다.
  - 검색 결과의 문장을 최대한 그대로 인용합니다.

  ### 검색 결과:
  '''

  instruction = instruction

  # 백터DB 쿼리
  query = instruction

  results = collection.query(
      query_embeddings=model.encode([query]),
      n_results=n_similer
  )

  # 결과를 문자열로 변환
  results_content = "\n".join([result["chunk"] for result in results['metadatas'][0]])

  messages = [
      {"role": "system", "content": f"{PROMPT}\n\n\n### 질문:{results_content}"},
      {"role": "user", "content": f"{instruction}"}
  ]

  return messages


In [ ]:
LLama3 = None

In [ ]:
"mistralai/Mistral-Large-Instruct-2407"
"rtzr/ko-gemma-2-9b-it"
"meta-llama/Meta-Llama-3.1-8B-Instruct"
"meta-llama/Meta-Llama-3.1-70B-Instruct"
"Nacho-Cola/llama3.1_report_model_0812" #파인튜닝

In [ ]:
%pwd

'/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회'

In [ ]:
# @title Inference
hugging_face_token = "token" # @param {type:"string"}
csv_path = "train.csv" # @param {type:"string"}
llama_model_id = "Nacho-Cola/llama3.1_report_model_0813" # @param {type:"string"}
similar_n = 5 # @param {type:"integer"}
model_temperature = 0.9 # @param {type:"number"}


# Hugging Face에 로그인
login(token=hugging_face_token)

# LLama3 모델을 로드합니다.
if LLama3 is None:
    LLAMA_3_MODEL_ID = llama_model_id

    # 4비트 양자화 설정
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

    # 토크나이저 및 모델 로드
    tokenizer = AutoTokenizer.from_pretrained(LLAMA_3_MODEL_ID)
    LLama3 = AutoModelForCausalLM.from_pretrained(
        LLAMA_3_MODEL_ID,
        torch_dtype="auto",
        quantization_config=quantization_config,
        device_map="auto",
    )
    LLama3.eval()  # 모델을 평가 모드로 설정

# CSV 파일 로드
df = pd.read_csv(csv_path)

# 고유한 PDF 경로 추출
pdfs = df['Source_path'].unique()

# 질문 리스트 초기화
Questions = []
for ind, pdf in enumerate(pdfs):
    Questions.append({
        'index': 'pdf_' + str(ind),
        'Source': pdf,
        'Question': df.loc[df['Source_path'] == pdf]['Question']
    })

answers = []  # 답변 리스트 초기화

# 각 질문에 대해 처리
for Question in tqdm(Questions):
    documents = load_pdf_file(Question['Source'])  # PDF 파일 로드
    split_docs = split_documents(documents)  # 문서 분할

    meta_dict = []  # 메타데이터 저장 리스트
    page_content = []  # 페이지 내용 저장 리스트
    for doc in split_docs:
        meta_dict.append(doc.metadata)  # 메타데이터 추가
        page_content.append(doc.page_content)  # 페이지 내용 추가

    embeddings = get_embedding_function(page_content)  # 임베딩 생성

    chroma_client = chromadb.Client()  # ChromaDB 클라이언트 초기화
    collection = chroma_client.create_collection(extract_alphanumeric(Question['index']+Question['Source']))  # 컬렉션 생성

    metadatas = []  # 메타데이터 저장 리스트
    ids = []  # ID 저장 리스트

    for i, article in tqdm(enumerate(split_docs)):  # 분할된 문서 반복
        metadatas.append({
            'source': article.metadata['source'],  # 소스 추가
            'page': article.metadata['page'],      # 페이지 추가
            'chunk': article.page_content           # 문서 내용 추가
        })
        ids.append(str(i))  # ID 추가

    embeddings_list = embeddings.tolist()  # 임베딩 리스트로 변환

    # ChromaDB에 데이터 추가
    collection.add(
        embeddings=embeddings_list,
        metadatas=metadatas,
        ids=ids
    )

    prompts = []  # 프롬프트 저장 리스트
    for instruction in Question['Question']:
        prompts.append(make_prompt(instruction, n_similer=similar_n))  # 프롬프트 생성

    for message in tqdm(prompts):  # 프롬프트 반복
        input_ids = tokenizer.apply_chat_template(
            message,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(LLama3.device)  # 입력 ID 생성

        terminators = [
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]  # 종료 토큰 설정

        # 모델을 통해 응답 생성
        outputs = LLama3.generate(
            input_ids,
            max_new_tokens=512,
            eos_token_id=terminators,
            do_sample=True,
            temperature=model_temperature,
            top_p=0.9,
            repetition_penalty=1.1,
        )

        print('\n'+message[1]['content'])
        response = outputs[0][input_ids.shape[-1]:]  # 응답 추출
        answer = tokenizer.decode(response, skip_special_tokens=True)
        answers.append(answer)  # 응답 디코딩하여 추가

        print(answer)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/104k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.24G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Supplied state dict for model.layers.22.mlp.down_proj.weight does not contain `bitsandbytes__*` and possibly other `quantized_stats` components.

In [ ]:
# @title Inference
similar_n = 3 # @param {type:"integer"}
model_temperature = 1 # @param {type:"number"}



answers = []  # 답변 리스트 초기화
for Question in tqdm(Questions):
  prompts = []  # 프롬프트 저장 리스트
  for instruction in Question['Question']:
    prompts.append(make_prompt(instruction, n_similer=similar_n))  # 프롬프트 생성

  for message in tqdm(prompts):  # 프롬프트 반복
    input_ids = tokenizer.apply_chat_template(
        message,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(LLama3.device)  # 입력 ID 생성

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]  # 종료 토큰 설정

    # 모델을 통해 응답 생성
    outputs = LLama3.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=model_temperature,
        top_p=0.9,
        repetition_penalty=1.1,
    )

    print('\n'+instruction)
    response = outputs[0][input_ids.shape[-1]:]  # 응답 추출
    answer = tokenizer.decode(response, skip_special_tokens=True)
    answers.append(answer)  # 응답 디코딩하여 추가

    print(answer)

  0%|          | 0/50 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
  0%|          | 0/16 [00:01<?, ?it/s]


KeyboardInterrupt: 

## F1-score

In [ ]:
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):

    #공백 제거
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())

    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

def calculate_average_f1_score(true_sentences, predicted_sentences):

    total_precision = 0
    total_recall = 0
    total_f1_score = 0

    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score

    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)

    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }


result = calculate_average_f1_score(df['Answer'], answers)
print(result)

{'average_precision': 0.44563658724562516, 'average_recall': 0.7840232023340656, 'average_f1_score': 0.5171466585380146}


In [ ]:
len(answers), len(df['Answer'])

(496, 496)

In [ ]:
answers

['2022년 혁신창업사업화자금(융자)의 예산은 2,300,000백만원입니다.',
 '중소벤처기업부의 혁신창업사업화자금(융자)의 사업목적은 다음과 같습니다: \n\n① 창업기반지원: 기술력과 사업성이 우수하고 미래 성장 가능성이 높은 중소벤처기업의 창업을 활성화하고 고용 창출을 도모하자는 것입니다. \n\n② 개발기술사업화: 중소기업이 보유한 우수 기술의 상징을 방지하고 개발 기술의 제품화•사업화를 촉진하여 기술기반 중소기업을 육성하기 위한 것입니다.',
 '중소벤처기업부의 혁신창업사업화자금(융자)의 사업근거는 다음과 같습니다: \n\n중소기업진흥에 관한 법률 제66조, 제67조, 제74조와 중소기업창업지원법 제35조가 근거입니다.',
 '2010년에는 재창업자금(실패 경영인에 대한 재기지원)을 신규로 지원했습니다.',
 '혁신창업사업화자금의 2020년 신규 지원된 자금은 미래기술육성자금 및 고성장촉진자금입니다.',
 '2015년입니다.',
 '일자리창출촉진자금이 창업기반지원과 신청 대상이 중복인 자금이며, 이 자금이 폐지된 연도는 2023년입니다.',
 '혁신창업사업화자금(융자) 사업은 중소벤처기업부가 중소기업진흥에 관한 법률 제66조, 제67조, 제74조와 중소기업창업지원법 제35조에 따라 시행하는 사업입니다.',
 '혁신창업사업화자금(융자)의 사업 집행절차는 다음과 같습니다. \n\n사업계획수립/공고 → 사전상담 및 신청‧접수 → 서류 및 현장실사 → 융자 실행 → 지원결정통보 → 평가 및 승인 → 융자(중소기업 → 중진공 → 중기부, 은행).',
 '부모급여 지원 사업의 목적은 아동 발달의 특성에 따라 영아기 돌봄을 두텁게 지원하기 위함이며, 출산 및 양육으로 손실되는 소득을 보전하고 주 양육者の 직접 돌봄이 중요한 아동 발달을 지원하기 위해 부모급여 지급하는 것입니다.',
 '부모급여(영아수당)의 2024년 확정된 예산은 2,888,694백만원입니다.',
 '아동수당법 제4조의 제5항에 근거하여 부모급여 지원 사업이 추진되고 있습니다.',
 '아동수당

In [ ]:
submission = pd.read_csv('sample_submission.csv')

In [ ]:
submission['Answer'] = answers

In [ ]:
submission

,SAMPLE_ID,Answer
0,TEST_000,"2022년 혁신창업사업화자금(융자)의 예산은 2,300,000백만원입니다."
1,TEST_001,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 기술력과 사업성이 우수하고 미래...
2,TEST_002,"혁신창업사업화자금(융자) 사업은 중소기업진흥에 관한 법률 제66조, 제67조, 제7..."
3,TEST_003,2010년에는 재창업자금(실패 경영인에 대한 재기지원)이 신규 지원되었습니다.
4,TEST_004,"2020년에는 미래기술육성자금, 고성장촉진자금이 신규 지원되었습니다."
...,...,...
93,TEST_093,공적보증채무는 보증인이 기타 공공부문과 민간부문 단위 기존 채무의 원리금 상환을 보...
94,TEST_094,미래사회보장급여에 대한 순의무는 계약상의 부채는 아니며 미래 발생 비용으로 여겨지는...
95,TEST_095,국가결산보고서는 국가회계기준에 따라 우발부채를 의무 이행을 위해 자원이 유출될 가능...
96,TEST_096,"우발부채는 미래에 특정 사건이 발생하면 발생하는 잠재적 의무 또는 부채로, 현재 의..."


In [ ]:
submission

,SAMPLE_ID,Answer
0,TEST_000,"2022년 혁신창업사업화자금(융자)의 예산은 2,300백만원 입니다."
1,TEST_001,창업기반지원 사업은 기술력과 사업성이 우수하고 미래 성장 가능성이 높은 중소벤처기업...
2,TEST_002,"중소기업진흥에 관한 법률 제66조, 제67조, 제74조에 근거하고 있습니다."
3,TEST_003,2010년에는 재창업자금(실패 경영인에 대한 재기지원) 신규 지원이 되었습니다.
4,TEST_004,"2020년에는 미래기술육성자금, 고성장촉진자금이 신규 지원되었습니다."
...,...,...
93,TEST_093,공적보증채무는 보증인이 기타 공공부문과 민간부문 단위 기존 채무의 원리금 상환을 보...
94,TEST_094,"미래사회보장급여에 대한 순의무는 계약상의 부채는 아니며, 미래 발생 비용으로 여겨지..."
95,TEST_095,국가결산보고서에서는 우발부채를 '우발사항 및 약정사항'이라는 주석에 공시하며 재무제...
96,TEST_096,"우발부채는 미래에 특정 사건이 발생하면 발생할 수 있는 잠재적 의무 또는 부채로, ..."


In [ ]:
submission.to_csv('submission_gemma2_9b_it_tem_09_sim_5.csv', index=False)

In [ ]:
len(answers)

98

In [ ]:
result = pd.read_csv('submission.csv')

In [ ]:
result

,Unnamed: 0,SAMPLE_ID,Answer
0,0,TEST_000,"2022년 혁신창업사업화자금(융자)의 예산은 2,300,000 만원입니다."
1,1,TEST_001,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 창업기반 지원과 개발기술 사업화...
2,2,TEST_002,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 중소기업진흥에 관한 법률 제66...
3,3,TEST_003,2010년 신규 지원된 혁신창업사업화자금은 재창업자금(실패 경영인에 대한 재기지원)...
4,4,TEST_004,혁신창업사업화자금 중 2020년에 신규 지원된 자금으로 미래기술육성자금 및 고성장촉...
...,...,...,...
93,93,TEST_093,"공적보증채무와 다른 일회성 보증은 융자 및 기타 채무상품 보증, 기타 일회성 보증으..."
94,94,TEST_094,"미래사회보장급여에 대한 순의무는 계약상의 부채가 아니며, 미래 발생 비용으로 여겨지..."
95,95,TEST_095,국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 서로 ...
96,96,TEST_096,"우발부채는 미래의 다양한 의제의무를 포함하는 재정적 의무를 부담하는 부채로, 국제기..."


In [ ]:
data = load_pdf_file('/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회/test_source/국토교통부_행복주택출자.pdf')
ls = split_documents(data)

/usr/local/lib/python3.10/dist-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [ ]:
ls

[Document(metadata={'source': '/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회/test_source/국토교통부_행복주택출자.pdf', 'page': 0}, page_content='사 업 명\r\n(26) 행복주택출자 (1440-408)\r\n□ 사업 코드 정보\r\n구분 기금 소관 실국(기관) 계정 분야 부문\r\n코드 주택도시기금 국토교통부 주택토지실 주택계정\r\n080 088\r\n명칭 사회복지 주택\r\n구분 프로그램 단위사업 세부사업\r\n코드 1400 1440 408\r\n명칭 임대주택지원(출자) 임대주택지원 행복주택출자\r\n□ 사업 성격 (공통요구자료 Ⅱ-1 작성유의사항 4. 참조, 해당하는 사항에 “○” 표시)\r\n신규 계속 완료 예비타당성\r\n실시여부\r\n총사업비\r\n관리대상\r\n총액계상\r\n예산사업\r\n사업소관 변경정보\r\n2023예산 시 소관\r\nO 국토교통부\r\n□ 사업 지원 형태 및 지원율 (최소한 한 개는 반드시 선택하시오. 해당사항에 O 표시)\r\n직접 출자 출연 보조 융자 국고보조율(%) 융자율 (%)\r\nO O 30\r\n□ 사업 담당자\r\n사업명 구분\r\n행복주택\r\n출자 국토교통부\r\n실·국·과(팀) 과 장 사무관 주무관\r\n주택토지실 김광림 곽상훈 임건혁 주거복지정책관\r\n공공주택정책과 044-201-4539 044-201-4519 044-201-3362\r\n가. 지출계획 총괄표 \r\n(단위: 백만원, %)\r\n목명 2022년 \r\n결산\r\n2023년 계획 2024년 증감\r\n당초 (B-A) (A) 수정 정부안 국회확정(B) (B-A)/A\r\n출자금 등 564,342 684,607 461,607 532,913 528,783 △155,824 △22.8'),
 Document(metadata={'source': '/content/drive/My